In [1]:
## Following along with https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
## Using "Multivariate Multi-Step LSTM Models" Section

## Helpful links
## https://stackoverflow.com/questions/54354467/how-to-shape-large-dataframe-for-pythons-keras-lstm
## https://towardsdatascience.com/how-to-reshape-data-and-do-regression-for-time-series-using-lstm-133dad96cd00

In [76]:
import pandas as pd
import numpy as np
from pathlib import Path

from numpy import array
from numpy import hstack 

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

import hvplot.pandas

ModuleNotFoundError: No module named 'hvplot'

In [3]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [4]:
## Load a dataset 
symbol = 'GME'

import_path = Path('../FilesExport_Complete_DFs_TI_noShift/'+symbol+'_TI_DF_no_shift.pkl')
import_data = load_obj(import_path)
dataframe = import_data[symbol]

#import_data

In [12]:
#dataframe.head()

In [5]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

# ## Fit scaler
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

# scaler.fit(y_train)
# y_train = scaler.transform(y_train)
# y_test = scaler.transform(y_test)

In [45]:
## Try changing length 
# len(dataframe) ## 1491

split = 1400*0.7
int(split)

dataframe = dataframe.tail(1400)

In [46]:
df = dataframe.copy()
df_close = df[['close']]
df = df.drop(columns=['close','adjClose'])

In [47]:
X = df
y = df_close
#X.head()

In [48]:
i = 0
for col in X.columns:
    i += 1
print(i) ## 20 features 

20


In [49]:
## Split data
split = int(0.7 * len(X))
x_train = X[: split]
x_test = X[split:]
y_train = y[: split]
y_test = y[split:]

In [50]:
len(x_train)

980

In [9]:
## Scale data 
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
scaler_y = StandardScaler()

x_train_sc = scaler_x.fit_transform(x_train)
x_test_sc = scaler_x.transform(x_test)
y_train_sc = scaler_y.fit_transform(y_train)
y_test_sc = scaler_y.transform(y_test)

In [22]:
len(x_train_sc)

980

In [34]:
x_train_sc[0]

array([ 1.39777052,  1.38392754,  1.38695085, -0.43909168, -0.43909168,
       -1.14313672, -0.53211119,  1.37284382, -0.53211119, -0.01092989,
       -0.55213896, -0.22397642, -0.49751295, -0.41346139, -0.21884317,
       -0.38773641,  0.5345931 , -1.25439832,  1.30502731,  0.12478208])

In [19]:
20860/3

6953.333333333333

In [63]:
##  
num_steps = 5
input_dim = 20
output_dim = 1


x_train_shaped = np.reshape(x_train_sc,newshape=(-1,num_steps,input_dim))

## It looks like a math thing - the length of the data likely needs to divide evenly (no decimals) for shaping
## to occur properly. Changing the length of the dataframe and data got the data to reshape. 

In [65]:
## Retry shaping 

## training set
x_train_shaped = np.reshape(x_train_sc, newshape=(-1, num_steps, input_dim))
y_train_shaped = np.reshape(y_train_sc, newshape=(-1, num_steps, output_dim))
assert x_train_shaped.shape[0] == y_train_shaped.shape[0]
## test set
x_test_shaped = np.reshape(x_test_sc, newshape=(-1, num_steps, input_dim))
y_test_shaped = np.reshape(y_test_sc, newshape=(-1, num_steps, output_dim))
assert x_test_shaped.shape[0] == y_test_shaped.shape[0]

In [66]:
## Attempt an LSTM Model 

In [68]:
model = Sequential()
model.add(LSTM(units=20, activation='tanh', input_shape=(num_steps, input_dim), return_sequences=True))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=1, activation='linear'))
#adam = optimizers.Adam(lr=0.001)
model.compile(optimizer='adam', loss='mse')

In [69]:
model.fit(x_train_shaped, y_train_shaped, epochs=10)
test_predict = model.predict(x_test_shaped)

Epoch 1/10
7/7 [==============================] - 1s 2ms/step - loss: 0.9736
Epoch 2/10
7/7 [==============================] - 0s 2ms/step - loss: 0.7769
Epoch 3/10
7/7 [==============================] - 0s 2ms/step - loss: 0.6121
Epoch 4/10
7/7 [==============================] - 0s 2ms/step - loss: 0.4784
Epoch 5/10
7/7 [==============================] - 0s 2ms/step - loss: 0.3640
Epoch 6/10
7/7 [==============================] - 0s 2ms/step - loss: 0.2842
Epoch 7/10
7/7 [==============================] - 0s 2ms/step - loss: 0.2218
Epoch 8/10
7/7 [==============================] - 0s 1ms/step - loss: 0.1765
Epoch 9/10
7/7 [==============================] - 0s 2ms/step - loss: 0.1468
Epoch 10/10
7/7 [==============================] - 0s 3ms/step - loss: 0.1273


array([[[-0.6062573 ],
        [-1.0017625 ],
        [-1.2364255 ],
        [-1.2728112 ],
        [-1.3753318 ]],

       [[-0.6832534 ],
        [-1.1252311 ],
        [-1.3892292 ],
        [-1.4467386 ],
        [-1.5621356 ]],

       [[-0.6576036 ],
        [-1.0434911 ],
        [-1.2659826 ],
        [-1.4139084 ],
        [-1.5708264 ]],

       [[-0.55141664],
        [-1.0432931 ],
        [-1.3177234 ],
        [-1.5093421 ],
        [-1.6149112 ]],

       [[-0.5610428 ],
        [-1.0876297 ],
        [-1.5174108 ],
        [-1.6000757 ],
        [-1.6677377 ]],

       [[-0.8862    ],
        [-1.2311109 ],
        [-1.3987849 ],
        [-1.4611009 ],
        [-1.5249156 ]],

       [[-0.53065217],
        [-0.87342656],
        [-1.1731519 ],
        [-1.27575   ],
        [-1.3365434 ]],

       [[-0.6744242 ],
        [-0.9801203 ],
        [-1.249131  ],
        [-1.345533  ],
        [-1.4585258 ]],

       [[-0.511462  ],
        [-0.9985682 ],
        [-1.238428